In [ ]:
import pandas as pd # Дата фреймы 
import json  # Работа с json'ами
import numpy as np # Для отделения от выборки 10 процентов и  Модуль для того чтоб мы посчитали логарифм
import matplotlib.pyplot as plt # Для боксплотов
from pandas.io.json import json_normalize  # Модуль для придания json'ам человеческого вида
from sklearn.ensemble import GradientBoostingRegressor # Градиентный бустинг
from sklearn.ensemble import RandomForestRegressor # Рандомный лес
from sklearn.svm import SVR # Эпсилон-опорная векторная регрессия
from sklearn.linear_model import LinearRegression # Линейная регрессия
from sklearn.model_selection import train_test_split # Разбиение на тестовую и обучающую
from sklearn.model_selection import KFold # Крос валидация 
from sklearn.model_selection import cross_val_score # Оценки валидаций

In [ ]:
JSON = ['device', 'geoNetwork', 'totals', 'trafficSource'] # Колонки с json
transact = pd.read_csv(r"train2.csv", sep = ",",  dtype = {'fullVisitorId': 'str'},
                   converters={column: json.loads for column in JSON}, 
                   nrows = None)  # Грузим сет 
for column in JSON:
        column_as_df = json_normalize(transact[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        transact = transact.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)  # Преобразуем json
cut = np.random.rand(len(transact)) < 0.01 # Перемешиваем строки
work_transact = transact[cut] # Срез с 0 до длины сета * 10%  и приведенной к целому числу
work_transact.head()
work_transact.info()

In [ ]:
result_work_transact = pd.DataFrame()
work_transact['totals.transactionRevenue'].fillna(0, inplace=True)
result_work_transact['Revenue'] = work_transact['totals.transactionRevenue'].astype(float)# Заполняем пустоты в выручке нулями, потом приводим к числам и считаем логарифм
result_work_transact['Revenue'] = np.log1p(result_work_transact['Revenue'].values) # Заполняем пустоты в выручке нулями, потом приводим к числам и считаем логарифм

result_work_transact['SocialEngagement'] = work_transact['socialEngagementType'] # Берем из оригинального сета только нужные параметры и желательно чтоб булики или цифры или можно было привести к буликам
result_work_transact = result_work_transact.replace({'SocialEngagement': {'Engaged': True, 'Not Socially Engaged': False}})
result_work_transact['isMobile'] = work_transact['device.isMobile']
result_work_transact['VisitNum'] = work_transact['visitNumber'].astype(float)
result_work_transact['Visits'] = work_transact['totals.visits'].astype(float)
result_work_transact['Hits'] = work_transact['totals.hits'].astype(float)
result_work_transact['PageViews'] = work_transact['totals.pageviews'].astype(float)
work_transact['totals.pageviews'].fillna(0, inplace=True)
work_transact['totals.bounces'].fillna(0, inplace=True)
work_transact['totals.newVisits'].fillna(0, inplace=True)
result_work_transact['PageViews'] = work_transact['totals.pageviews'].astype(float)
result_work_transact['Bonuces'] = work_transact['totals.bounces'].astype(float)
result_work_transact['NewVisists'] = work_transact['totals.newVisits'].astype(float)

result_work_transact.head()
result_work_transact.info()

In [ ]:
Y = result_work_transact['Revenue']  # Отделяем от выборки значение зависимой переменной
Train = result_work_transact.drop(columns=['Revenue'], axis=1)
Xtrn, Xtest, Ytrn, Ytest = train_test_split(Train, Y, test_size = 0.2) # Разбиваем выборку на тестовую и обучающую


In [ ]:
Bosting = GradientBoostingRegressor()
scores = np.array([])
for i in range(10):  # Тут теперь 10 разбиений
       fold = KFold(n_splits=5, shuffle=True)
       scores_on_this_split = cross_val_score(
               estimator = Bosting, X=Xtrn, y=Ytrn,
               cv=fold)
       scores = np.append(scores, scores_on_this_split)
fig1, ax1 = plt.subplots()
ax1.set_title('Бустинг')
ax1.boxplot(scores)

In [ ]:
Forest = RandomForestRegressor(n_estimators = 20)
scores = np.array([])
for i in range(10):  # Тут теперь 10 разбиений
       fold = KFold(n_splits=5, shuffle=True)
       scores_on_this_split = cross_val_score(
               estimator = Forest, X=Xtrn, y=Ytrn,
               cv=fold)
       scores = np.append(scores, scores_on_this_split)
fig1, ax1 = plt.subplots()
ax1.set_title('Лес')
ax1.boxplot(scores)

In [ ]:
SVR = SVR(kernel='rbf', C=1e3, gamma = 0.1)
scores = np.array([])
for i in range(2):  # Тут теперь 10 разбиений
       fold = KFold(n_splits=5, shuffle=True)
       scores_on_this_split = cross_val_score(
               estimator = SVR, X=Xtrn, y=Ytrn,
               cv=fold)
       scores = np.append(scores, scores_on_this_split)
fig1, ax1 = plt.subplots()
ax1.set_title('Векторная регрессия')
ax1.boxplot(scores)

In [ ]:
LinearRegression = LinearRegression()
scores = np.array([])
for i in range(10):  # Тут теперь 10 разбиений
       fold = KFold(n_splits=5, shuffle=True)
       scores_on_this_split = cross_val_score(
               estimator = LinearRegression, X=Xtrn, y=Ytrn,
               cv=fold)
       scores = np.append(scores, scores_on_this_split)
fig1, ax1 = plt.subplots()
ax1.set_title('Линейная регрессия')
ax1.boxplot(scores)